# Testing Vaadin Flow apps with jUnit

> tags: []

# Testing Vaadin Flow apps with jUnit

Learn how to run unit and integration tests on Vaadin Flow apps.


1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  Unit- en integratietests

-   [Unittests maken en uitvoeren voor eenvoudige UI-logica](https://vaadin.com/docs/latest/tutorial/unit-and-integration-testing/#creating-and-running-unit-tests-for-simple-ui-logic)
-   [Integratietests maken en uitvoeren voor geavanceerdere UI-logica](https://vaadin.com/docs/latest/tutorial/unit-and-integration-testing/#creating-and-running-integration-tests-for-more-advanced-ui-logic)

Het is een gebruikelijke best practice om zo min mogelijk code in één test te testen. Op deze manier, als het misgaat, falen alleen relevante tests. Voor UI-testen zijn er drie hoofdbenaderingen:

-   Unit tests: voor eenvoudige UI logica.
    
-   Integratietests: voor meer geavanceerde UI-logica.
    
-   End-to-end tests: om te testen wat de gebruiker ziet.
    

U kunt unit- en integratietests zelfstandig uitvoeren, dat wil zeggen zonder externe afhankelijkheden, zoals een actieve server of database.

End-to-end tests vereisen dat de toepassing wordt geïmplementeerd en worden uitgevoerd in een browservenster om een daadwerkelijke gebruiker te simuleren.

In dit hoofdstuk schrijf en voer je unit- en integratietests uit. De tutorial behandelt end-to-end tests in het volgende hoofdstuk.

## [](https://vaadin.com/docs/latest/tutorial/unit-and-integration-testing/#creating-and-running-unit-tests-for-simple-ui-logic)Unittests maken en uitvoeren voor eenvoudige UI-logica

De meest minimale manier van testen is het maken van een gewone Java-eenheidstest. Dit werkt alleen met UI-klassen zonder afhankelijkheden, zonder automatische bedrading, enz. Voor de , kunt u een eenheidstest maken om te controleren of de formuliervelden correct zijn ingevuld, op basis van de opgegeven boon.`ContactForm`

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Plaats tests in de juiste map</p><p><span _mstmutation="1" _msthash="2705772" _msttexthash="24103235">Alle testklassen moeten in de testmap gaan, . Besteed speciale aandacht aan de pakketnamen. Gebruik pakkettoegang voor klassevelden. Als de test niet in hetzelfde pakket zit als de klasse die u test, krijgt u fouten.</span><code>src/test/java</code></p></td></tr></tbody></table>

Maak een nieuwe map, . Klik in IntelliJ met de rechtermuisknop op de map en selecteer **Directory markeren als > Test Sources Root**. Maak in de nieuwe map een nieuw pakket en voeg een nieuw ContactFormTest.java bestand met de volgende code toe.`src/test/java``com.example.application.views.list`

```
package com.example.application.views.list;

import com.example.application.data.entity.Company;
import com.example.application.data.entity.Contact;
import com.example.application.data.entity.Status;
import org.junit.Assert;
import org.junit.Before;
import org.junit.Test;

import java.util.ArrayList;
import java.util.List;
import java.util.concurrent.atomic.AtomicReference;

public class ContactFormTest {
    private List<Company> companies;
    private List<Status> statuses;
    private Contact marcUsher;
    private Company company1;
    private Company company2;
    private Status status1;
    private Status status2;

    @Before 1
    public void setupData() {
        companies = new ArrayList<>();
        company1 = new Company();
        company1.setName("Vaadin Ltd");
        company2 = new Company();
        company2.setName("IT Mill");
        companies.add(company1);
        companies.add(company2);

        statuses = new ArrayList<>();
        status1 = new Status();
        status1.setName("Status 1");
        status2 = new Status();
        status2.setName("Status 2");
        statuses.add(status1);
        statuses.add(status2);

        marcUsher = new Contact();
        marcUsher.setFirstName("Marc");
        marcUsher.setLastName("Usher");
        marcUsher.setEmail("marc@usher.com");
        marcUsher.setStatus(status1);
        marcUsher.setCompany(company2);
    }
}
```

1.  The annotation adds dummy data that is used for testing. This method is executed before each method.`@Before``@Test`
    

Now, add a test method that uses :`ContactForm`

```
@Test
public void formFieldsPopulated() {
    ContactForm form = new ContactForm(companies, statuses);
    form.setContact(marcUsher); 1
    Assert.assertEquals("Marc", form.firstName.getValue());
    Assert.assertEquals("Usher", form.lastName.getValue());
    Assert.assertEquals("marc@usher.com", form.email.getValue());
    Assert.assertEquals(company2, form.company.getValue());
    Assert.assertEquals(status1, form.status.getValue()); 2
}
```

1.  Validates that the fields are populated correctly, by first initializing the contact form with some companies, and then setting a contact bean for the form.
    
2.  Uses standard JUnit methods to compare the values from the fields available through the instance:`assertEquals()``ContactForm`
    

Similarly, test the "save" functionality of .`ContactForm`

```
@Test
public void saveEventHasCorrectValues() {
    ContactForm form = new ContactForm(companies, statuses);
    Contact contact = new Contact();
    form.setContact(contact); 1
    form.firstName.setValue("John"); 2
    form.lastName.setValue("Doe");
    form.company.setValue(company1);
    form.email.setValue("john@doe.com");
    form.status.setValue(status2);

    AtomicReference<Contact> savedContactRef = new AtomicReference<>(null);
    form.addListener(ContactForm.SaveEvent.class, e -> {
        savedContactRef.set(e.getContact()); 3
    });
    form.save.click(); 4
    Contact savedContact = savedContactRef.get();

    Assert.assertEquals("John", savedContact.getFirstName()); 5
    Assert.assertEquals("Doe", savedContact.getLastName());
    Assert.assertEquals("john@doe.com", savedContact.getEmail());
    Assert.assertEquals(company1, savedContact.getCompany());
    Assert.assertEquals(status2, savedContact.getStatus());
}
```

1.  Initialize the form with an empty .`Contact`
    
2.  Populate values into the form.
    
3.  Capture the saved contact into an .`AtomicReference`
    
4.  Click the **save** button and read the saved contact.
    
5.  Once the event data is available, verify that the bean contains the expected values.
    

To run the unit test, right-click and select **Run 'ContactFormTest'**.`ContactFormTest`

![run unit test](https://vaadin.com/docs/latest/static/dbfb6c9a0d94b446c82274996f3b618f/03979/run-unit-test.png)

When the test finishes, you will see the results at the bottom of the IDE window in the test-runner panel. As you can see, both tests passed.

![tests passed](https://vaadin.com/docs/latest/static/37842caf58d511ea4184db98fda236b6/03979/unit-test-results.png)

## [](https://vaadin.com/docs/latest/tutorial/unit-and-integration-testing/#creating-and-running-integration-tests-for-more-advanced-ui-logic)Creating and Running Integration Tests for More Advanced UI Logic

To test a class that uses , a database, or any other feature provided by Spring Boot, you can no longer use plain JUnit tests. Instead, you can use the Spring Boot test runner. This does add a little overhead, but it makes more features available to your test.`@Autowire`

To set up a unit test for , create a new file, ListViewTest, in the package:`ListView``com.example.application.views.list`

```
package com.example.application.views.list;

import org.junit.Test;
import org.junit.runner.RunWith;
import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.boot.test.context.SpringBootTest;
import org.springframework.test.context.junit4.SpringRunner;

@RunWith(SpringRunner.class) 1
@SpringBootTest
public class ListViewTest {

    @Autowired
    private ListView listView;

    @Test
    public void formShownWhenContactSelected() {
    }
}
```

1.  The and annotations make sure that the Spring Boot application is initialized before the tests are run and allow you to use in the test.`@RunWith(SpringRunner.class)``@SpringBootTest``@Autowire`
    

In the class:`ListView`

1.  Add the Spring annotation to make it possible to it.`@Component``@Autowire`
    
2.  Also add to ensure that every test run gets a fresh instance.`@Scope("prototype")`
    

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>The annotation is not needed for normal application runs</p><p><span _mstmutation="1" _msthash="2705773" _msttexthash="12394408">You did not need to add the annotation for normal application usage, as all classes are automatically instantiated by Vaadin in a Spring-compatible way.</span><code>@Route</code></p></td></tr></tbody></table>

```
@Component
@Scope("prototype")
@Route(value = "", layout = MainLayout.class)
@PageTitle("Contacts | Vaadin CRM")
@PermitAll
public class ListView extends VerticalLayout {
    Grid<Contact> grid = new Grid<>(Contact.class);
    TextField filterText = new TextField();
    ContactForm form;
    CrmService service;

    // rest omitted
}
```

Right-click the package that contains both tests, and select Run tests in 'com.example.application.views.list'. You should see that both test classes run and result in three successful tests.

![run package tests](https://vaadin.com/docs/latest/static/f065164c1083a726fec8ecfdcf4281ae/03979/run-tests-in-package.png)

![three successful tests](https://vaadin.com/docs/latest/static/db56d3d8fba31a5569c8c2f21393eeb6/03979/three-tests-passed.png)

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Integration tests take longer to run</p><p><span _mstmutation="1" _msthash="2705774" _msttexthash="17321720">You probably noticed that running the tests the second time took much longer. This is the price of being able to use and other Spring features and can cost many seconds of startup time.</span><code>@Autowire</code></p><p _msthash="2705775" _msttexthash="1530490">Options for improving the startup time include:</p><div><ul><li><p><span _mstmutation="1" _msthash="3536078" _msttexthash="2868099">explicitly listing the needed dependencies in the annotation using </span><code>@SpringBootTest</code><code>classes={…}</code>;</p></li><li><p _msthash="3536079" _msttexthash="930384">mocking up parts of the application;</p></li><li><p _msthash="3536080" _msttexthash="3409055">using other advanced techniques which are out of the scope of this tutorial.</p></li></ul></div></td></tr></tbody></table>

You can now add the actual test implementation, which selects the first row in the grid and validates that this shows the form with the selected :`Contact`

```
@Test
public void formShownWhenContactSelected() {
    Grid<Contact> grid = listView.grid;
    Contact firstContact = getFirstItem(grid);

    ContactForm form = listView.form;

    Assert.assertFalse(form.isVisible());
    grid.asSingleSelect().setValue(firstContact);
    Assert.assertTrue(form.isVisible());
    Assert.assertEquals(firstContact.getFirstName(), form.firstName.getValue());
}

private Contact getFirstItem(Grid<Contact> grid) {
    return( (ListDataProvider<Contact>) grid.getDataProvider()).getItems().iterator().next();
}
```

The test verifies that the form logic works by:

-   asserting that the form is initially hidden;
    
-   selecting the first item in the grid and verifying that:
    
    -   the form is visible;
        
    -   the form is bound to the correct by ensuring that the right name is visible in the field.`Contact`
        
    

Rerun the tests. They should all pass.

You now know how to test the application logic both in isolation with unit tests and by injecting dependencies to test the integration between several components. In the next chapter, the tutorial covers how to test the entire application in the browser.

If your components depend on , , and similar, you may need to fake or mock the Vaadin environment for those tests to pass. For further information on how to achieve that, you can take a look at the [Karibu-Testing](https://github.com/mvysny/karibu-testing/) project.`UI.getCurrent()``UI.navigate()`

76D89DA1-B104-4745-8D51-9589846051C8

[TutorialInstallation and offline (PWA)](https://vaadin.com/docs/latest/tutorial/installing-and-offline-pwa)[TutorialEnd-to-end testing](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench)

**Was this page helpful?**  
Leave a comment or a question below. You can also join the [chat on Discord](https://discord.gg/MYFq5RTbBn) or [ask questions on StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
